# Bibliotecas

In [1]:
import sys

sys.path.append('/eos/user/t/thenriqu/Dark_Matter/LGBM_hhdm_analysis/')


import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hepherolib.analysis as ana

from lgbm.controllers_lgbm import LGBLearner, LGBModel

ana.start()
plt.style.use("default")

# Configuração

In [2]:
period = '17'
year_style = 2017
dataset_year = "2017"
basedir = '/eos/user/t/thenriqu/Dark_Matter/Amostras/hhdmAnalysis_deepJet_Regions/datasets'
dataset_name = basedir.split('/')[-2]

# Data folder
data_path = f"./data/{dataset_name}/{dataset_year}"

# Setup output folders
models_path = f"./models/{dataset_name}/{dataset_year}"
Path(models_path).mkdir(parents=True, exist_ok=True)

# Lendo os dados

In [3]:
model_name = "LGB_multi_signal"

X_train = pd.read_csv(f"{data_path}/{model_name}-train-data.csv")
X_test = pd.read_csv(f"{data_path}/{model_name}-test-data.csv")
W_train = X_train.pop("modelWeight").values
W_test = X_test.pop("modelWeight").values
Y_train = X_train.pop("Label").values
Y_test = X_test.pop("Label").values
evtWeight_train = X_train.pop("evtWeight").values
evtWeight_test = X_test.pop("evtWeight").values

In [4]:
X_train.head()

,LeadingLep_pt,LepLep_deltaM,LepLep_deltaR,LepLep_pt,MET_LepLep_Mt,MET_LepLep_deltaPhi,MET_pt,MT2LL,Nbjets,TrailingLep_pt
0,47.163464,0.235672,2.265808,67.92354,128.03561,2.862613,61.526054,61.06800,1,24.065823
1,172.587650,1.318451,1.578457,175.15870,188.94249,2.127670,66.668750,65.81853,2,22.481535
2,126.914690,12.223457,0.707707,216.75772,221.90184,3.097569,56.819534,109.99913,2,99.981910
3,84.078545,1.770401,1.512148,108.57854,143.58008,2.310987,56.695393,70.79929,1,42.847990
4,143.305710,0.598106,0.690091,255.85414,313.49194,2.549555,104.960400,154.72617,1,128.369500


In [5]:
W_train

array([5.99839959e-01, 3.03238276e-01, 1.56494986e-03, ...,
       5.20275446e+00, 1.85627215e-01, 6.08263914e+00])

In [6]:
Y_train

array([0, 0, 0, ..., 1, 0, 1])

# Procura por Hiperparametros

#### Usado pelo Macedo - LGBMClassifier

n_iter = 150
cv = 2
param_grid = {
    'learning_rate': [0.008, 0.0004],
    'n_estimators': [900, 1000],
    'num_leaves': [24,128,62],
    'min_child_samples':[100,400,800],
    'subsample':[1,0.9, 0.8],
    'colsample_bytree':[0.9,0.8,0.2],
    'objective': ['multiclass'],
    'metric': ['multi_logloss'],
    'num_class':[3],
    'boosting_typ’: [‘gbdt’]
    'feature_pre_filter': ['False']
}




#### Usado pelo Gabriel: XGBoostClassifier


params = {
    'n_estimators': [100, 500, 1000, 2000, 3000, 5000, 7000, 9000],
    'min_child_weight': range(1,10),
    'learning_rate': [1e-3, 1e-2, 1e-1, 0.3, 0.5, 0.7, 0.9, 1.],
    'subsample': np.arange(0.50, 1.01, 0.05),
    'colsample_bytree': np.arange(0.50, 1.01, 0.05),
    'max_depth': range(3,11),
    'eval_metric': ['logloss', 'error']
}






#### Usado pelo Thiago ( Eu ): vottingclassifier 


log_clf = LogisticRegression(solver = 'newton-cg', random_state = 42)
rnd_clf = XGBClassifier( objective = 'multiclass', n_estimators = 4000, learning_rate = 0.001, random_state = 42)
lgbm_clf = LGBMClassifier(objective = 'multiclass',  num_class = 3, n_estimators = 4000, metric = 'multi_logloss', learning_rate = 0.001, boosting_type = 'gbdt', random_state = 42)

voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('lgbm', lgbm_clf)],
    voting='soft')



In [7]:

params = {
    'n_estimators': [900, 1000, 5000, 7000, 9000],
    'min_child_weight': range(1,10),
    'min_child_samples': [100, 400, 800],
    'learning_rate': [0.001, 0.004, 0.008],
    'subsample': [1, 0.9, 0.8],
    'colsample_bytree': [0.9, 0.8, 0.2],
#     'colsample_bytree': np.arange(0.50, 1.01, 0.05),
#     'subsample': np.arange(0.50, 1.01, 0.05),
    'max_depth': range(3,11),
    'eval_metric': ['logloss', 'error']
}



learner = LGBLearner(X_train, Y_train, W_train, njobs=12)
res = learner.find_hyperparams(hyperparams_grid=params, n_splits=5, n_iter=30)


Fitting 5 folds for each of 30 candidates, totalling 150 fits


/cvmfs/sft.cern.ch/lcg/views/LCG_105a_swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 116919, number of negative: 1556251
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005004 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2303
[LightGBM] [Info] Number of data points in the train set: 1673170, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.666599 -> initscore=0.692843
[LightGBM] [

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 116920, number of negative: 1556251
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.474081 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2303
[LightGBM] [Info] Number of data points in the train set: 1673171, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.666618 -> initscore=0.692928
[LightGBM] [Info] Start training from score 0.692928
[LightGBM] [Warning] No

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 116919, number of negative: 1556252
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.593742 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2303
[LightGBM] [Info] Number of data points in the train set: 1673171, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.666935 -> initscore=0.694355
[LightGBM] [Info] Start training from score 0.694355
[LightGBM] [Warning] No

...

[CV 2/5] END colsample_bytree=0.8, eval_metric=logloss, learning_rate=0.008, max_depth=6, min_child_samples=800, min_child_weight=6, n_estimators=900, subsample=1;, score=0.470 total time=24.5min
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 116920, number of negative: 1556251
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.389325 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[CV 3/5] END colsample_bytree=0.9, eval_metric=logloss, learning_rate=0.004, max_depth=10, min_child_samples=800, min_child_weight=4, n_estimators=5000, subsample=1;, score=0.478 total time=129.4min
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 146149, number of negative: 1945314
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.072280 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2303
[LightGBM] [Info] Number of data points in the trai

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

# Salvando os Parametros

In [8]:
with open(f'{models_path}/{model_name}-bst-params.json', 'w') as f:
    json.dump(res['hyperparameters'], f, ensure_ascii=False, indent=4)

# Noficação Quando o Notebook terminar de compilar

In [9]:
from IPython.display import clear_output, display, HTML, Javascript

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Process completed!";
  window.speechSynthesis.speak(msg);
  alert("Process completed!")
"""))

<IPython.core.display.Javascript object>